# 0.Installing dependencies And Bring in Data 

In [10]:
!pip install tensorflow

In [11]:
!pip install pandas matplotlib sklearn


In [12]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [13]:
tf.__version__

'2.14.0'

In [14]:
path = os.path.join('jigsaw-toxic-comment-classification-challenge','train.csv','train.csv')
path # gives us the path of file

'jigsaw-toxic-comment-classification-challenge\\train.csv\\train.csv'

In [15]:
df = pd.read_csv(path)


In [16]:
df.head()
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [17]:
df.iloc[12]['comment_text']


'Hey... what is it..\n@ | talk .\nWhat is it... an exclusive group of some WP TALIBANS...who are good at destroying, self-appointed purist who GANG UP any one who asks them questions abt their ANTI-SOCIAL and DESTRUCTIVE (non)-contribution at WP?\n\nAsk Sityush to clean up his behavior than issue me nonsensical warnings...'

In [18]:
df[df.columns[2 :]].loc[0]

toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
Name: 0, dtype: int64

# 1.Data Preprocessing

### a.Tokenization

In [19]:
from tensorflow.keras.layers import TextVectorization

In [20]:
X = df['comment_text']
y = df[df.columns[2:]].values #.values converts the df to array of values

In [21]:
list1= X.values;
for x in list1:
    print(x)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [22]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [23]:
MAX_FEATURES = 200000 # number of word in the vocab 

In [24]:
vectorizer = TextVectorization(max_tokens = MAX_FEATURES , 
                              output_sequence_length = 1800,
                              output_mode ='int')
#Max_tokens = How many word we want in our vocabulary
# Maximum length of token = output_sequence_length
#output mode = interger mapping everyword to integer like  I -> 64 and like that

In [25]:
vectorizer

In [26]:
vectorizer.adapt(X.values)

In [27]:
vectorized_text = vectorizer(X.values)

In [28]:
len(vectorized_text)

159571

In [29]:
vectorized_text[:1799]

<tf.Tensor: shape=(1799, 1800), dtype=int64, numpy=
array([[ 645,   76,    2, ...,    0,    0,    0],
       [   1,   54, 2489, ...,    0,    0,    0],
       [ 425,  441,   70, ...,    0,    0,    0],
       ...,
       [  49,  447,  752, ...,    0,    0,    0],
       [  21,  582,    9, ...,    0,    0,    0],
       [ 616,   19,    6, ...,    0,    0,    0]], dtype=int64)>

In [30]:
len(vectorized_text)

159571

In [32]:
#MCSHBAP - map , cache ,shuffle , batch , prefetch 
#For Pipelining

dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps prevent bottelnecks


In [33]:
batch_X , batch_y = dataset.as_numpy_iterator().next()
X = pd.DataFrame(batch_X)
Y = pd.DataFrame(batch_y)
print(X.iloc[:])
print(Y.iloc[:])


      0     1      2     3     4      5     6      7     8      9     ...  \
0   180877   992      9  7957   199  64168  2103   7439  2417      5  ...   
1       21   467    425     7   118      3  7952     33     7     72  ...   
2        2     1   1464   160  2673     10  1397  34294     9  46806  ...   
3      863  1646  28701     4  6712      8    25   8622     3    149  ...   
4      387     5      7   385   349    173     8     39    79    835  ...   
5      139     7   1025   118   139    200  2298   2372  1025   2372  ...   
6      349    86   1540   179     8    558     7      3   839      7  ...   
7       14    28    340     3    17    137    41     20    49    261  ...   
8        2  1084     43    58    66     13   255    169   247   6174  ...   
9       94    61     90    19   845   4151    34   2858    61     47  ...   
10     458     5   1198     2    24    492    43   1960   677    726  ...   
11      67     1   1558    10     2   2228     4      6   589  26552  ...   

In [34]:
batch_X.shape # 16 samples/batches of len 1800 each

(16, 1800)

In [35]:
batch_y.shape

(16, 6)

In [36]:
len(dataset)

9974

In [37]:
# 70 per of dataset to train data
train = dataset.take(int(len(dataset)*.7)) 

# Skips first 70 % of dataset and take 20 % from remaining 30%
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))

# skips 90% of data and take 10 % remaining for testing use
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [38]:
train_generator = train.as_numpy_iterator()

In [39]:
train_generator.next()

(array([[188072,      1,     18, ...,      0,      0,      0],
        [ 96950,  19715,    485, ...,      0,      0,      0],
        [    82,      2,   2980, ...,      0,      0,      0],
        ...,
        [   171,     46,     55, ...,      0,      0,      0],
        [    23,      8,    215, ...,      0,      0,      0],
        [  1122,      8,    590, ...,      0,      0,      0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

# 2. Create Sequential Model


In [40]:
#using sequential api -> Allow us
from tensorflow.keras.models import Sequential
#Fastest and easiest 
from tensorflow.keras.layers import LSTM , Dropout , Bidirectional , Dense , Embedding


In [41]:
model = Sequential()
#Creating layers
model.add(Embedding(MAX_FEATURES+1 , 32)) # 32 length 
#Adding a Bidirection LSTM layer
#32 Different LSTM units/ nueron activation of tanh
model.add(Bidirectional(LSTM(32,activation ='tanh')))
# Feature Extractor fully connected layers(Dense Layers)
model.add(Dense(128 , activation = 'relu'))
model.add(Dense(256 , activation = 'relu'))
model.add(Dense(128 , activation = 'relu'))
#Final layers Connect the output
model.add(Dense(  6 , activation = 'sigmoid'))





In [42]:
model.compile(loss = 'BinaryCrossentropy' , optimizer = 'Adam')

In [43]:
train

<_TakeDataset element_spec=(TensorSpec(shape=(None, 1800), dtype=tf.int64, name=None), TensorSpec(shape=(None, 6), dtype=tf.int64, name=None))>

In [44]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirection  (None, 64)                16640     
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [ ]:
history = model.fit(train , epochs = 10 , validation_data = val)

Epoch 1/10
2080/6981 [=======>......................] - ETA: 40:38 - loss: 0.0861

In [ ]:
# import matplotlib import pyplot as plt

# 3. Making Predictions


In [ ]:
input_text = vectorizer('You freaking suck!')

In [ ]:
batch_X , batch_y = test.as_numpy_iterator().next() 

In [ ]:
(model.predict(batch_X) > 0.5).astype(int) 

In [ ]:
batch_y

# 5. Model Evaluation

In [ ]:
from tensorflow.keras.metrics import Precision , Recall , CategoricalAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    #Unpacking the batch
    X_true , y_true = batch
    
    #Making the prediction
    yhat = model.predict(X_true)
    
    #Flatten the prediction
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true , yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

In [ ]:
print(f"Precision : {pre.result().numpy()}  \nRecall : {re.result().numpy()} \nAccuracy : { acc.result().numpy()} ")

In [ ]:
fn = (2 * pre.result().numpy() * re.result().numpy()) / (re.result().numpy() + pre.result().numpy())
fn

In [ ]:
import gradio as gr

In [ ]:
model = tf.keras.models.load_model('toxicity.h5')

In [ ]:
input_str = vectorizer('For fucks sake I will kill you.')
res = model.predict(np.expand_dims(input_str,0))

In [ ]:
res

In [ ]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    result = model.predict(vectorized_comment)
    
    text = ""
    
    for idx,col in enumerate(df.columns[2:]):
        text += '{} : {}\n'.format(col , result[0][idx] > 0.3)
        
    return text
    

In [ ]:
with gr.Blocks() as interface:
    Comment = gr.Textbox(label = "Comment" , placeholder = "Enter comment here")
    Results = gr.Textbox(label = "Results")
    check_toxicity = gr.Button("Check Toxicity")
    check_toxicity.click(fn = score_comment,
                         inputs = Comment,
                         outputs = Results,
                         api_name = "greet")

In [ ]:
interface.launch(share = True)

In [ ]:
# Gradio

In [ ]:
import tensorflow as tf


In [ ]:
tf.__version__


In [ ]:
nvcc --version
